In [3]:
# Import standard Python modules
from collections import OrderedDict, defaultdict

# Import openalea packages
from openalea.deploy.shared_data import shared_data

from openalea.mtg import *
from openalea.mtg.algo import orders
from openalea.mtg import stat

import openalea.plantgl.all as pgl

# Import strawberry modules
import openalea.strawberry

# rename Rules_production to geometry
from openalea.strawberry import Rules_production
# Replace Visualisation by visualisation 
#from openalea.strawberry import Visualization
from openalea.strawberry import analysis

# TODO: Move functions into ???
#from openalea.strawberry import variables

NOT using graph editor observer No module named grapheditor


In [4]:
%gui qt4

files = shared_data(openalea.strawberry).glob('*.mtg')

genotypes = dict((f.namebase, f) for f in files)
print(genotypes.keys())

DIPLOIDS = ['Nils_2']
OCTOPLOIDS = ['Gariguette_2']

diploids = dict((k,f) for k,f in genotypes.items() if k in DIPLOIDS)
print(diploids)

octoploids = dict((k,f) for k,f in genotypes.items() if k in OCTOPLOIDS)
print(octoploids)


[u'Sicile_New', u'Gariguette_NEW', u'Darselect_New', u'Nils', u'Ciflorette', u'Nils_New', u'Clery_New', u'Darselect', u'Capriss_New', u'Gariguette_2', u'Gariguette', u'Cir107_NEW', u'Nils2.63.39', u'Nils_2', u'Capriss', u'Clery', u'Ciflorette_New', u'Cir107']
{u'Nils_2': Path(u'c:\\users\\mlabadie\\appdata\\local\\continuum\\miniconda2\\envs\\strawberry\\lib\\site-packages\\openalea.strawberry-1.0.0-py2.7.egg\\share\\data\\Nils_2.mtg')}
{u'Gariguette_2': Path(u'c:\\users\\mlabadie\\appdata\\local\\continuum\\miniconda2\\envs\\strawberry\\lib\\site-packages\\openalea.strawberry-1.0.0-py2.7.egg\\share\\data\\Gariguette_2.mtg')}


# Load one octoploid and one diploid  genotype 

A set of strawberry plant architecture has been measured and is stored in an MTG.

We select on octoploid and one diploid genotype for architectural analysis.


In [5]:
di_genotype = 'Nils_2'
oct_genotype = 'Gariguette_2'

gdi = MTG(diploids[di_genotype])
print('Diploid  : ', str(gdi))

goct = MTG(octoploids[oct_genotype])
print('Octoploid  : ', str(goct))


replace all the date format by -
('Diploid  : ', 'MTG : nb_vertices=3623, nb_scales=4')
replace all the date format by -
('Octoploid  : ', 'MTG : nb_vertices=6519, nb_scales=4')


In [6]:
# Diploid
nb_plants = gdi.nb_vertices(scale=1)
print(nb_plants)

# Octoploid
nb_plants = goct.nb_vertices(scale=1)
print(nb_plants)


112
54


In [7]:
# Run the notebook on Diploid or Octoploid
IS_DIPLOID = False

if IS_DIPLOID:
    g = gdi
else:
    g = goct



print(g.property_names())
convert = dict(Stade='Stade', 
               Fleurs_ouverte='FLWRNUMBER_OPEN',
               Fleurs_avorte='FLWRNUMBER_ABORTED',
               Fleurs_total='FLWRNUMBER',
               date='Sample_date',
               Plante='Plant_ID',
              )

if not IS_DIPLOID:
    g.properties()['Sample_date'] = g.property('Architecture_date')

def dates_diploid():
    l = '11-05-2016', '17-05-2016', '01-06-2016', '20-06-2016'
    return dict(zip(l, range(1, len(l)+1)))

def dates_octoploid():
    d = {'10-12-2014':1,'07-01-2015':2,'15-02-2015':3,'02-03-2015':4,'03-04-2015':5,'27-05-2015':6,'04-12-2014':1,'13-02-2015':3,
         '30-03-2015':5,'08-01-2015':2,'11-02-2015':3,'04-03-2015':4,'02-04-2015':5,'20-05-2015':6,'09-01-2015':2,'06-03-2015':4,
         '12-02-2015':3,'19-05-2015':6}
    return d

def dates():
    if IS_DIPLOID:
        return dates_diploid()
    else: 
        return dates_octoploid()


def property(g, name):
    """ We can change the name of the MTG properties without changing the code"""
    return g.property(convert.get(name, name))
    

['LFTWD_LEFT', 'Genotype', 'Foliar_type', 'LFTAR_CENTRAL', 'LFTLG_RIGHT', 'SAMPLING', 'Experiment_name', 'edge_type', 'LFTWD_CENTRAL', 'Architecture_date', 'Sample_date', 'PETLG', 'COMMENT', 'index', 'label', 'LFTAR_LEFT', 'LFTAR_RIGHT', '_line', 'Stade', 'LFTLG_LEFT', 'LFTWD_RIGHT', 'FLWRNUMBER_OPEN', 'DBI', 'LFAR', 'INFLOLG', 'Plant_ID', 'FLWRNUMBER', 'LFTLG_CENTRAL', 'FLWRNUMBER_ABORTED', 'FLWRNUMBER_CLOSED']


# Sequence extraction according to Modules 
## For each module (scale = 2) qui porte un F qui se termine par HT, ht, bt


### Index
- real Order (Index) for recouvrant extraction
- Apparent Order for non recouvrant extraction

### Variables:
 - nb_visible_leaves : count F label in components
 - nb_foliar_primordia: count f si le module commence par F
 - nb_total_leaves: sum(F+f) du module
 - Stolon Numbers: count s label in components and attach at the order -1 
 - nb_open_flowers: g.property('Fleurs_ouverte') at scale 3
 - nb_aborted_flowers: g.property('Fleurs_avorte') at scale 3
 - nb_total_flowers : g.property('Fleurs_total') at scale 3
 - No_vegetative_bud: count bt if g.property(Stade) == none or 17 or 18 or 19
 - No_iniated_bud: count bt if g.property(Stade) == A
 - No_Floral_bud: count ht
 - No_Inflorescence: count HT
 
###Variable qualitative
 - type of crown
     - label == T (1) : principal crown
     - parent(component_roots()[0]) : if successor() == F : branch_crown elif in (HT, ht, bt) extension crown
 - inflorescence_type:
     - vegetative (1): label==bt g.property(Stade)== none or 17 or 18 or 19
     - initiated (2): label== bt if g.property(Stade) == A
     - floral (3): label== ht or HT

### General Variables
- Genotype: g.property('Genotype') at scale 1
- Date: g.property('date') at scale 1
- Plante Number: g.property('plante') at scale 1

In [8]:
# apparent_module: all module which contains a visible leaf F
def visible_modules(g=g):
    return [v for v in g.vertices_iter(scale=2) if g.label(g.component_roots_iter(v).next()) == 'F']

In [9]:
# add property in MTG, apparant module (True) or not
modules = visible_modules()
_visible = {}
for m in modules:
    _visible[m] = True
g.properties()['visible'] = _visible


In [10]:
"""Index

TODO: Replace dict by functions with specific traversal.

"""
#order scale 2: order at module scale
orders = algo.orders(g, scale=2)

"""Variable quantitative"""
#function which count all F
def nb_visible_leaves(vid, g=g):
    return sum(1 for cid in g.components(vid) if g.label(cid)=='F')

#function which count all f
def nb_foliar_primordia(vid, g=g):
    return sum(1 for cid in g.components(vid) if g.label(cid)=='f')

#function which count all f+F
def nb_total_leaves(vid, g=g):
    return sum(1 for cid in g.components(vid) if g.label(cid) in ('f', 'F'))

""" nb_stolon"""
#function count stolon
def nb_stolons(v, g=g):
    def nb_stolon(vid, g=g):
        return sum(1 for cid in g.components(vid) if g.label(cid)=='s') 
    return sum(nb_stolon(ch) for ch in g.children(v))

#function return number of open flower
def nb_open_flowers(vid, g=g):
    flowers = property(g, 'Fleurs_ouverte')
    return sum( flowers.get(cid,0) for cid in g.components(vid) if g.label(cid) in ('ht', 'HT'))

# function return number of aborted flower
def nb_aborted_flowers(vid, g=g):
    flowers = property(g, 'Fleurs_aborted')
    return sum( flowers.get(cid,0) for cid in g.components(vid) if g.label(cid) in ('ht', 'HT'))

# function return number of total flower
def nb_total_flowers(vid, g=g):
    flowers = property(g, 'Fleurs_total')
    return sum( flowers.get(cid,0) for cid in g.components(vid) if g.label(cid) in ('ht', 'HT'))


"""Return the No vegetative bud
    
Algorithm:
if labet is bt then stage is 17,18,19 or None
count number of bt and attach at the parent order
    """
# function return number of vegetative buds
def nb_vegetative_buds(vid, g=g):
    stages= property(g, 'Stade')

    def nb_vegetative(v):
        cid = g.component_roots(v)[0]
        
        return sum(1 for cid in g.components(v) if g.label(cid)=='bt' and stages.get(cid) in (None,'17','18','19'))
    
    return sum(nb_vegetative(ch) for ch in g.children(vid))

 
""" Return the No initiated bud"""

def nb_initiated_buds(vid, g=g):
    
    stages= property(g, 'Stade')
    
    def nb_init(v):
        return sum(1 for cid in g.components(v) if (g.label(cid)=='bt') and (stages.get(cid)=='A'))

    return sum(nb_init(ch) for ch in g.children(vid))


""" Return the No Floral bud"""
def nb_floral_buds (vid, g=g):
    visibles = property(g, 'visible')
    def nb_floral(v):
        return sum(1 for cid in g.components(v) if g.label(cid)=="ht" )
    return sum(nb_floral(ch) for ch in g.children(vid) if ch not in visibles)


""" Qualitative variables"""
def type_of_crown(vid, g=g):
    """ Returns the type of crown.
    
    Definition of type of crown (1, 2, 3):
     - principal crown (1): label == T 
     - branch_crown (3)
         parent(component_roots()[0]) : if successor() == F  
     - extension_crown (2): contains(HT, ht, bt) 
     - error (4)
     
    """
    if g.label(vid) == 'T':
        return 1
    else:
        cid = g.component_roots_iter(vid).next()
        pid = g.parent(cid)
        sid = g.Successor(pid)
        #print sid
        if g.label(sid) in ('F', 'f'):
            return 3
        elif g.label(sid) in ('bt', 'ht', 'HT'):
            return 2
        else:
            # ERROR !!!
            print g[cid], g[g.complex_at_scale(cid, scale=1)]
            return 4
            
def Crown_status(vid, g=g):
    """ Returns the type of inflorescence
    
    :Algorithms:
    if label is bt then
        - if stage is 17, 18, 19 or None, => vegetative (1)
        - if stage is A => initiated (2)
        - if stage is other => non defined (pourri, avorté, coupé) (-1)
     - Terminal vegetative bud (1): label==bt g.property(Stade)== none or 17 or 18 or 19
     - Terminal initiated bud (2): label== bt if g.property(Stade) == A
     - Terminal Floral bud (3): label==ht
     - Inflorescence Terminal (4): label== HT
     - runner (5): label = s
     
    """
    stages = property(g,'Stade')
    # select s, ht, HT et bt
    for cid in g.components(vid):
        if g.label(cid) in ('s', 'ht', 'HT', 'bt'):
            label = g.label(cid)
            if label == 'ht':
                return 3
            elif label == 'HT':
                return 4
            elif label == 'bt':
                stage = stages.get(cid)
                if stage == 'A':
                    return 2
                elif stage in (None, '17', '18', '19'):
                    return 1
            elif label == 's':
                return 5
            return -1

def nb_inflorescence (Vid, g):
    return sum(1 for cid in g.components(Vid) if g.label(cid)=='HT')
    
def genotype(vid, g=g):
    d = {'Capriss':4, 'Ciflorette':2, 'Cir107':6, 'Clery':3, 'Darselect':5, 'Gariguette':1,
         'Nils': 1, }
    
    cpx = g.complex_at_scale(vid, scale=1)
    _genotype = property(g, 'Genotype')[cpx]
    return d[_genotype]

def plant(vid, g=g):
    
    cpx = g.complex_at_scale(vid, scale=1)
    return property(g, 'Plante')[cpx]

def date(vid, g=g):
    d = dates()
    
    cpx = g.complex_at_scale(vid, scale=1)
    _date = property(g, 'date')[cpx]
    return(d[_date])

In [11]:
# Extract sequences

#seqs = stat.extract_extremities(g, scale=2)
#seqs = stat.extract_axes(g, scale=2)
def is_extremities(g, v):
    visibles = property(g, 'visible')
    ext = [c for c in g.children(v) if c in visibles and not is_axis_root(g, c)]
    return True if not ext else False

def extract_extremities(g, modules):
    """ Extract sequences that connect from leaves modules to the root ones of each plant. """
    leaves = [m for m in modules if is_extremities(g, m)]
    seqs = [list(reversed([vid for vid in algo.ancestors(g,lid)])) for lid in leaves]
    return seqs

def is_axis_root(g, vid):
    cid = g.component_roots_iter(vid).next()
    pid = g.parent(cid)
    sid = g.Successor(pid)
    if g.label(sid) not in ('bt', 'ht', 'HT'):
        return True
    else:
        return False
    

def apparent_axis(g, vid):
    visibles = property(g, 'visible')
    v = vid
    while v is not None:
        yield v
        vtx = v; v = None
        for vid in g.children(vtx):
            if (vid in visibles) and (not is_axis_root(g, vid)):
                v = vid
            
def extract_apparent_axes(g, modules):
    # Extract all the vertices with edge_type == '+'
    vids = [m for m in modules if is_axis_root(g, m)]
    
    seqs = [ list(apparent_axis(g, vid)) for vid in vids]
    return seqs


In [21]:
seqs1 = extract_apparent_axes(g, modules)
seqs2= extract_extremities(g,modules)
seqs = seqs1
len (seqs)
seqs

[[2],
 [74],
 [147],
 [264],
 [327],
 [383],
 [445],
 [505],
 [562],
 [636],
 [674],
 [692],
 [714],
 [817],
 [914, 1003],
 [948],
 [980],
 [1051],
 [1053],
 [1202],
 [1322],
 [1427],
 [1534],
 [1621],
 [1710, 1920],
 [1742],
 [1759],
 [1804],
 [1853],
 [1898],
 [1948, 2038],
 [1950],
 [2017],
 [2068, 2139],
 [2118],
 [2168, 2218],
 [2239, 2417],
 [2263],
 [2290],
 [2384],
 [2442, 2574],
 [2489],
 [2547],
 [2604, 2656],
 [2679, 2766],
 [2743],
 [2791, 2865],
 [2838],
 [2897, 2957],
 [2927],
 [2988, 3051, 3058],
 [3021],
 [3076, 3149, 3163],
 [3129, 3134],
 [3195, 3294, 3304],
 [3262, 3269],
 [3325, 3409, 3417],
 [3374, 3386],
 [3442, 3520],
 [3484],
 [3554, 3570, 3595, 3605],
 [3628, 3695, 3709],
 [3733, 3853, 3861],
 [3735, 3748],
 [3832],
 [3880, 4075, 4094],
 [3885],
 [3898, 3925],
 [3911],
 [3961, 3981],
 [4030, 4043],
 [4082],
 [4125, 4208, 4215, 4225],
 [4174, 4184],
 [4243, 4296],
 [4277],
 [4311, 4443, 4455],
 [4313, 4339],
 [4315],
 [4407, 4417],
 [4477, 4525, 4532],
 [4508],


In [17]:
# Index: order
VertexIdentifiers = [['-'.join(str(orders[j]) for j in seq[0:i+1]) for i, vid in enumerate(seq)] for seq in seqs]

# Variables
variables = OrderedDict()
variables['nb_visible_leaves'] = nb_visible_leaves # Nombre de feuille developpe
variables['nb_foliar_primordia'] = nb_foliar_primordia #Nombre de primordia foliaire
variables['nb_total_leaves'] = nb_total_leaves #Nombre total de feuille
variables['nb_open_flowers'] = nb_open_flowers #Nombre de fleurs ouverte
variables['nb_aborted_flowers'] = nb_aborted_flowers #Nombre de fleurs avorté
variables['nb_total_flowers'] = nb_total_flowers #Nombre total de Fleurs
variables['vegetative_bud'] = nb_vegetative_buds
variables['Initiated_bud']= nb_initiated_buds
variables['floral_bud']= nb_floral_buds
variables['stolons']= nb_stolons
variables['type_of_crown'] = type_of_crown # Type de crowns (Primary Crown:1, Branch crown:2 extension crown:3)
variables['Crown_status'] = Crown_status # Status du coeurs ("vegeatatif:1, Iniated: 2, Floral: 3)


variables['genotype'] = genotype # TODO
variables['date'] = date # TODO
variables['plant'] = plant

In [18]:
variables

OrderedDict([('nb_visible_leaves', <function __main__.nb_visible_leaves>),
             ('nb_foliar_primordia', <function __main__.nb_foliar_primordia>),
             ('nb_total_leaves', <function __main__.nb_total_leaves>),
             ('nb_open_flowers', <function __main__.nb_open_flowers>),
             ('nb_aborted_flowers', <function __main__.nb_aborted_flowers>),
             ('nb_total_flowers', <function __main__.nb_total_flowers>),
             ('vegetative_bud', <function __main__.nb_vegetative_buds>),
             ('Initiated_bud', <function __main__.nb_initiated_buds>),
             ('floral_bud', <function __main__.nb_floral_buds>),
             ('stolons', <function __main__.nb_stolons>),
             ('type_of_crown', <function __main__.type_of_crown>),
             ('Crown_status', <function __main__.Crown_status>),
             ('genotype', <function __main__.genotype>),
             ('date', <function __main__.date>),
             ('plant', <function __main__.plant>)

In [20]:
variable_names = variables.keys()
print variable_names

final_seqs = [[[f(vid) for f in variables.values()] for vid in seq] for seq in seqs]

final_seqs


['nb_visible_leaves', 'nb_foliar_primordia', 'nb_total_leaves', 'nb_open_flowers', 'nb_aborted_flowers', 'nb_total_flowers', 'vegetative_bud', 'Initiated_bud', 'floral_bud', 'stolons', 'type_of_crown', 'Crown_status', 'genotype', 'date', 'plant']


[[[8, 4, 12, 0, 0, 0, 1, 3, 7, 1, 1, 3, 1, 1, 1]],
 [[8, 4, 12, 0, 0, 0, 4, 3, 4, 1, 1, 3, 1, 1, 2]],
 [[11, 3, 14, 0, 0, 0, 3, 1, 8, 2, 1, 3, 1, 1, 3]],
 [[8, 3, 11, 0, 0, 0, 6, 0, 5, 0, 1, 3, 1, 1, 4]],
 [[6, 4, 10, 0, 0, 0, 5, 2, 2, 1, 1, 3, 1, 1, 5]],
 [[7, 4, 11, 0, 0, 0, 3, 2, 5, 1, 1, 3, 1, 1, 6]],
 [[7, 4, 11, 0, 0, 0, 7, 1, 2, 1, 1, 3, 1, 1, 7]],
 [[6, 3, 9, 0, 0, 0, 1, 3, 4, 1, 1, 3, 1, 1, 8]],
 [[7, 3, 10, 0, 0, 0, 4, 0, 5, 1, 1, 3, 1, 1, 9]],
 [[11, 8, 19, 0, 0, 0, 3, 2, 10, 1, 1, 3, 1, 2, 1]],
 [[2, 3, 5, 0, 0, 0, 5, 0, 0, 0, 3, 3, 1, 2, 1]],
 [[1, 4, 5, 0, 0, 0, 3, 0, 2, 0, 3, 3, 1, 2, 1]],
 [[1, 4, 5, 0, 0, 0, 3, 1, 1, 0, 3, 3, 1, 2, 1]],
 [[11, 4, 15, 0, 0, 0, 2, 2, 9, 2, 1, 3, 1, 2, 2]],
 [[7, 0, 7, 0, 0, 0, 1, 0, 3, 0, 1, 4, 1, 2, 3],
  [5, 1, 6, 0, 0, 0, 2, 0, 4, 0, 2, 3, 1, 2, 3]],
 [[3, 1, 4, 0, 0, 0, 2, 0, 2, 0, 3, 3, 1, 2, 3]],
 [[2, 4, 6, 0, 0, 0, 5, 0, 1, 0, 3, 3, 1, 2, 3]],
 [[11, 0, 11, 0, 0, 0, 0, 0, 10, 0, 1, 4, 1, 2, 4]],
 [[3, 3, 6, 0, 0, 0, 5, 1, 0, 0, 3

In [29]:
sequence_txt = analysis.write_sequences(final_seqs, variable_names, VertexIdentifiers)

In [15]:
# Check variables....
print sequence_txt

INDEX_PARAMETER : TIME   # vertex_id

15 VARIABLES

VARIABLE 1: INT  # nb_visible_leaves
VARIABLE 2: INT  # nb_foliar_primordia
VARIABLE 3: INT  # nb_total_leaves
VARIABLE 4: INT  # nb_open_flowers
VARIABLE 5: INT  # nb_aborted_flowers
VARIABLE 6: INT  # nb_total_flowers
VARIABLE 7: INT  # vegetative_bud
VARIABLE 8: INT  # Initiated_bud
VARIABLE 9: INT  # floral_bud
VARIABLE 10: INT  # stolons
VARIABLE 11: INT  # type_of_crown
VARIABLE 12: INT  # Crown_status
VARIABLE 13: INT  # genotype
VARIABLE 14: INT  # date
VARIABLE 15: INT  # plant


#Index	nb_visible_leaves	nb_foliar_primordia	nb_total_leaves	nb_open_flowers	nb_aborted_flowers	nb_total_flowers	vegetative_bud	Initiated_bud	floral_bud	stolons	type_of_crown	Crown_status	genotype	date	plant
0	8	4	12	0	0	0	1	3	7	1	1	3	1	1	1		#

0	8	4	12	0	0	0	4	3	4	1	1	3	1	1	2		#

0	11	3	14	0	0	0	3	1	8	2	1	3	1	1	3		#

0	8	3	11	0	0	0	6	0	5	0	1	3	1	1	4		#

0	6	4	10	0	0	0	5	2	2	1	1	3	1	1	5		#

0	7	4	11	0	0	0	3	2	5	1	1	3	1	1	6		#

0	7	4	11	0	0	0	7	1	2	1	

# Extraction at Plant scale

## Variables

* Plant
* Genotype
* date
* nb_total_leaves
* nb_total_flowers
* stolons
* order_max
* nb_ramification

In [16]:
plant_variables = OrderedDict()
# plant_variables['Genotype'] = genotype 
# plant_variables['date'] = date 
# plant_variables['plant'] = plant
plant_variables['nb_total_leaves'] = nb_total_leaves #Nombre total de feuille
plant_variables['nb_total_flowers'] = nb_total_flowers #Nombre total de Fleurs
plant_variables['stolons']= nb_stolons
#plant_variables['type_of_crown'] = type_of_crown # Type de crowns (Primary Crown:1, Branch crown:2 extension crown:3)


In [17]:
print plant_variables

OrderedDict([('nb_total_leaves', <function nb_total_leaves at 0x000000000A2B37B8>), ('nb_total_flowers', <function nb_total_flowers at 0x000000000A2B3D68>), ('stolons', <function nb_stolons at 0x000000000A2B3908>)])


In [18]:
import pandas as pd

plant_ids = g.vertices(scale=1)

plant_df = OrderedDict()
# for name in ('Genotype', 'date', 'plant'):
#     plant_df[name] = [plant_variables[name](pid) for pid in plant_ids]
plant_df['Genotype'] = [genotype(pid) for pid in plant_ids]
plant_df['date'] = [date(pid) for pid in plant_ids]
plant_df['plant'] = [plant(pid) for pid in plant_ids]

visibles = property(g, 'visible')

for name in (plant_variables):
    f = plant_variables[name]
    plant_df[name] = [sum(f(v) for v in g.components(pid) if v in visibles) for pid in plant_ids]

plant_df['order_max'] = [max(orders[v] for v in g.components(pid) if v in visibles) for pid in plant_ids]
plant_df['nb_ramifications'] = [sum(1 for v in g.components(pid) if (type_of_crown(v)==3 and v in visibles)) for pid in plant_ids]
plant_df['vid'] = plant_ids

df = pd.DataFrame(plant_df)
df

,Genotype,date,plant,nb_total_leaves,nb_total_flowers,stolons,order_max,nb_ramifications,vid
0,1,1,1,12,0,1,0,0,1
1,1,1,2,12,0,1,0,0,73
2,1,1,3,14,0,2,0,0,146
3,1,1,4,11,0,0,0,0,263
4,1,1,5,10,0,1,0,0,326
5,1,1,6,11,0,1,0,0,382
6,1,1,7,11,0,1,0,0,444
7,1,1,8,9,0,1,0,0,504
8,1,1,9,10,0,1,0,0,561
9,1,2,1,34,0,1,1,3,635


In [29]:
def median_individuals(df):
    indices = []
    for gd, dataf in df.groupby(["Genotype","date"]):
        geno, date = gd
        dg = dataf[df.columns[3:-1]]
        s=((dg-dg.median()).abs()/(dg-dg.median()).abs().mean()).sum(axis=1)
        indices.append(s.idxmin())
        
    return df.iloc[indices]

median_individuals(df)

,Genotype,date,plant,nb_total_leaves,nb_total_flowers,stolons,order_max,nb_ramifications,vid
5,1,1,6,11,0,1,0,0,382
14,1,2,6,11,18,1,0,0,1321
25,1,3,8,17,31,0,1,1,2678
29,1,4,3,20,42,0,2,1,3075
41,1,5,6,24,49,1,2,2,4556
45,1,6,1,35,52,3,4,2,5182


In [ ]:
indices = []
for gd, dataf in df.groupby(["Genotype","date"]):
    geno, date = gd
    dg = dataf[df.columns[3:-1]]
    s=((dg-dg.median()).abs()/(dg-dg.median()).abs().mean()).sum(axis=1)
    indices.append(s.idxmin())
    df["dist"]=s

    print indices

In [ ]:
res = []
for gd, dataf in df.groupby(["Genotype","date"]):
    geno, date = gd
    dg = dataf[df.columns[3:]]
    s=((dg-dg.median()).abs()/(dg-dg.median()).abs().mean()).sum(axis=1)
    dg["dist"]=s
    res.append((geno, date, s.idxmin(), s.min()))

print res

In [ ]:
indices=[5, 14, 25]

df.iloc[indices]

In [ ]:
df[df.index==45].plant

In [ ]:
df2.get_group((1L, 1L))

In [ ]:
l=[]
grouped = df.groupby(["Genotype","date"])
for g in grouped.groups:
    dg = grouped.get_group(g)[grouped.get_group(g).columns[3:]]
    l.append(((dg-dg.median()).abs()/(dg-dg.median()).abs().mean()).sum(axis=1))

In [ ]:
pd.DataFrame(l)

In [ ]:
f = plant_variables["nb_total_leaves"]
a = [f(v) for v in g.components(1) ]
plant_ids
print g.components(1)
set(x for v in g.components(1)for x in g.components(v))
f(2), f(4)

# Extraction at Phytomer scale
## 1.Index
Index is node on apparent trunk axis

## 2. Qualitative variable
On each nodes identify the type of ramification:
ramif-type: 
- if is invisible modules==F take form of: 
    - stolon (s): 1, 
    - vegetative bud(bt, at stage None, 17,18,19):2,
    - initiated bud, (bt, at stage A):3, 
    - floral bud(ht):4 
- if is visible module==T: 
    -branch crown (complex ramification):5

## 3. Variables
if branch crown(5):
- on principal path:
    - No. modules
    - No. developed leaves (F)
    - No. primordia leaves (f)
    - No. total leaves (f+F)
    - No. open flower, g.property("open_flower")
    - No. aborded flower, g.property("aborded_flower")
    - No. total flower, g.property("total flower")
- on whole branch crown:
    - No. branch crown carried sum(No. visible modules if modules is_axis_root==T )
    - No. extension crown carried sum(No. visible module if modules is_axis_root==F) 
    - No. total modules carried 
        - sum(No.visible modules)
    - No. developed leaves carried 
            sum(for vid in g if g.label=="F" and if visible module==T)
    - No. primordia leaves carried sum(f, if visible module==T)
    - No. total leaves carried sum(F+f, if visible module==T)
    - No. vegetative buds carried sum(bt at stage None, 17,18,19 if visible modules ==F)
    - No. initiated buds carried sum(bt at stage A if visible modules == F)
    - No. floral buds carried sum(ht if visible modules==F)
    - No. Inflorescence carried sum(HT if visible module==T)
    - No. open flowers carried sum(g.property("open_flowers"), if visible modules==T)
    - No. aborded flowers carried sum(g.property("aborded_flower"if visible module==T)
    - No. total flowers carried sum(g.property("total_total flowers" if visible module==T)
    - No. stolon carried sum(s, if visible module ==F)
 

In [ ]:
from itertools import chain
from openalea.mtg import algo, traversal

In [ ]:
# Extract the first module of each plant at scale 2
roots = g.component_roots_at_scale(g.root, scale=2)
# assert set(g.label(r) for r in roots) == set(['T']) 

trunks = [ list(chain(*[(v for v in algo.axis(g,m, scale=3) if g.label(v) in ('F', 'f')) for m in apparent_axis(g, r)])) for r in roots]
# print trunk

# define the apparent order on the MTG
vertex_ids = [[str(rank) for rank, v in enumerate(trunk)] for trunk in trunks]

# extract a sequence of F for each module

In [ ]:
def branching(g, v):
    sid = g.Successor(v)
    if g.label(sid) in ('bt', 'ht', 'HT'):
        return None
    
    kids = [c for c in g.children(v) if g.edge_type(c) =='+' and g.label(c) == 'F' ]
    if not kids:
        return None
    
    return kids[0]

def branching_type(vid, g=g):
    """ Returns the type of branching
    
    :Algorithms:
    
    if module is visible:
        - branch crown (complex ramification):6
        - inflorescence : 7

    if module is invisible:
        - stolon (s): 1,
        - vegetative bud(bt, at stage None, 17,18,19):2,
        - initiated bud, (bt, at stage A):3,
        - aborted or roten or dried bud: 4
        - floral bud(ht):5

     
    """

    cpx = g.complex(vid)
    nxid = g.node(cpx) 
    if nxid.visible:
        if g.label(vid) == 'HT':
            return 7
        else:
            return 6
    
    # select s, ht, et bt
    for cid in nxid.components():
        label = cid.label
        if label in ('s', 'ht', 'bt'):
            if label == 's':
                return 1
            elif label == 'bt':
                stage = cid.Stade
                if stage in (None, '17', '18', '19'):
                    return 2
                elif stage == 'A':
                    return 3
                elif stage in ('pourri', 'aborted', 'dried'):
                    return 4
            elif label  == 'ht':
                return 5
    else:
        print 'ERROR: ', cpx, nxid.complex().Genotype, nxid.properties()

#[[branching(g, v) for v in trunk] for trunk in trunks]

# Extract on a vertex the tree of modules
def module_tree(v, g):
    visibles = g.property('visible')
    cpx = g.complex(v)
    return [m for m in traversal.pre_order2(g, cpx) if m in visibles]


In [ ]:
def module_axis(v, g=g):
    cpx = g.complex(v)
    return list(apparent_axis(g, cpx))

def nb_modules_axis(v, g=g):
    return len(module_axis(v, g=g))

def nb_visible_leaves_axis(v, g=g):
    return sum(nb_visible_leaves(m, g) for m in module_axis(v, g=g))

def nb_foliar_primordia_axis(v, g=g):
    return sum(nb_foliar_primordia(m, g) for m in module_axis(v, g=g))

def nb_total_leaves_axis(v, g=g):
    return sum(nb_total_leaves(m, g) for m in module_axis(v, g=g))

def nb_open_flowers_axis(v, g=g):
    return sum(nb_open_flowers(m, g) for m in module_axis(v, g=g))

def nb_aborted_flowers_axis(v, g=g):
    return sum(nb_aborted_flowers(m, g) for m in module_axis(v, g=g))

def nb_total_flowers_axis(v, g=g):
    return sum(nb_total_flowers(m, g) for m in module_axis(v, g=g))

def nb_vegetative_buds_axis(v, g=g):
    return sum(nb_vegetative_buds(m, g) for m in module_axis(v, g=g))

def nb_initiated_buds_axis(v, g=g):
    return sum(nb_initiated_buds(m, g) for m in module_axis(v, g=g))

def nb_floral_buds_axis(v, g=g):
    return sum(nb_floral_buds(m, g) for m in module_axis(v, g=g))

def nb_stolons_axis(v, g=g):
    return sum(nb_stolons(m, g) for m in module_axis(v, g=g))


####################################################
def nb_total_module_tree(v, g=g):
    return len(module_tree(v, g))

def nb_branching_tree(v, g=g):
    return sum(1 for m in module_tree(v, g) if is_axis_root(g, m))

def nb_extension_tree(v,g=g):
    return sum(1 for m in module_tree(v,g) if not is_axis_root(g, m))

def nb_visible_leaves_tree(v, g=g):
    return sum(nb_visible_leaves(m,g) for m in module_tree(v, g))

def nb_primordia_leaves_trees(v, g=g):
    return sum(nb_foliar_primordia(m,g) for m in module_tree(v, g))

def nb_total_leaves_trees(v, g=g):
    return sum(nb_total_leaves(m, g) for m in module_tree(v, g))

def nb_vegetative_buds_tree(v, g=g):
    return sum(nb_vegetative_buds(m, g) for m in module_tree(v, g))

def nb_initiated_buds_tree(v, g=g):
    return sum(nb_initiated_buds(m, g) for m in module_tree(v, g))

def nb_floral_buds_tree(v, g=g):
    return sum(nb_floral_buds(m, g) for m in module_tree(v, g))

def nb_inflorescences_tree(v, g=g):
    return sum(nb_inflorescences(m, g) for m in module_tree(v, g))

def nb_open_flowers_tree(v, g=g):
    return sum(nb_open_flowers(m, g) for m in module_tree(v, g))

def nb_aborted_flowers_tree(v , g=g):
    return sum(nb_aborted_flowers(m, g=g) for m in module_tree(v, g))

def nb_total_flowers_tree(v, g=g):
    return sum(nb_total_flowers(m, g=g) for m in module_tree(v, g))

def nb_stolon_tree(v, g=g):
    return sum(nb_stolons(m, g) for m in module_tree(v, g))

def nb_inflorescences_tree(v, g=g):
    return sum(nb_inflorescence (m, g) for m in module_tree(v, g))

##################################################
def genotype_axis(v, g=g):
    return genotype(g.complex(v), g)
def date_axis(v, g=g):
    return date(g.complex(v), g)
def plant_axis(v, g=g):
    return plant(g.complex(v), g)
# test
#trunk = trunks[1]
#[nb_stolon_tree(branching(g, v)) for v in trunk if branching(g, v)]


In [ ]:
# Variables
variables = OrderedDict()

# on principal ramification path
#variables["branching_type"]= branching_type
variables['nb_module_axis'] = nb_modules_axis
variables['nb_visible_leaves_axis'] = nb_visible_leaves_axis # Nombre de feuille developpe
variables['nb_foliar_primordia_axis'] = nb_foliar_primordia_axis #Nombre de primordia foliaire
variables['nb_total_leaves_axis'] = nb_total_leaves_axis #Nombre total de feuille
variables['nb_vegetative_buds_axis']= nb_vegetative_buds_axis #
variables['nb_initiated_buds_axis']= nb_initiated_buds_axis
variables['nb_floral_buds_axis']= nb_floral_buds_axis
variables['nb_open_flowers_axis'] = nb_open_flowers_axis #Nombre de fleurs ouverte
variables['nb_aborted_flowers_axis'] = nb_aborted_flowers_axis #Nombre de fleurs avorté
variables['nb_total_flowers_axis'] = nb_total_flowers_axis #Nombre total de Fleurs
variables['nb_stolons_axis'] = nb_stolons_axis

#on whole ramification 
variables["nb_branch_crown_tree"]= nb_branching_tree # sum(No. visible modules if modules is_axis_root )
variables["nb_extenstion_crown_tree"]= nb_extension_tree # sum(No. visible module if modules nit is_axis_root)
variables["nb_total module_tree"]= nb_total_module_tree # sum(visible modules)
variables["nb_visible_leaves_tree"]= nb_visible_leaves_tree # sum(F, if visible module==T)
variables["nb_primordia_leaves_trees"]=nb_primordia_leaves_trees #sum(f, if visible module==T)
variables["nb_total_leaves_trees"]=nb_total_leaves_trees #sum(F & f , if visible module==T)
variables["nb_vegetative_buds_tree"]= nb_vegetative_buds_tree# sum(bt if stages in (None,'17','18','19') and visible module==F)
variables["nb_initiated_buds_tree"]= nb_initiated_buds_tree# sum(bt if stages=='A' and visible module==F)
variables["nb_floral_buds_tree"]= nb_floral_buds_tree# sum(ht if visible module==F)
variables["nb_inflorescence_tree"]= nb_inflorescences_tree# sum(HT if visible module==T)
variables["nb_open_flowers_tree"]= nb_open_flowers_tree# sum(g.property('Fleurs_ouverte') if visible module==T)
variables["nb_aborted_flowers_tree"]= nb_aborted_flowers_tree# sum(g.property('Fleurs_aborted') if visible module==T)
variables["nb_total_flowers_tree"]= nb_total_flowers_tree# sum(g.property('Fleurs_total') if visible module==T)
variables["nb_stolon_tree"]= nb_stolon_tree# sum(s if visible modules ==F)

# add supplemental variables
#variables No. Inflorescence
#variables No. Inflorescence crown bearing by ramification
#variables No Vegetative crown bearing by ramification
#variables sum initiated crown bearing by ramification
#variable sum floral crown bearing by ramification

#Global variable
global_variables = OrderedDict()
global_variables['genotype'] = genotype_axis # TODO
global_variables['date'] = date_axis # TODO
global_variables['plant'] = plant_axis

# Extract variables fo Inflorescences
inflo_variables = OrderedDict()
inflo_variables['nb_total_flowers_axis'] = nb_total_flowers 
inflo_variables['nb_open_flowers_axis'] = nb_open_flowers #Nombre de fleurs ouverte
inflo_variables['nb_aborted_flowers_axis'] = nb_aborted_flowers #Nombre de fleurs avorté



In [ ]:
final_seqs = []
# add if not ramif:
def is_inflo(g, vid):
    return True if g.label(vid) =='HT' else False

def filter_hamp_branch_crown(g, vid):
    """Extract first ramification node or inflo. """
    inflo = [c for c in g.children(vid) if is_inflo(g, c)]
    if inflo:
        return inflo[0]
    
    ramif = [c for c in g.children(vid) if (g.edge_type(c) =='+')]
    if not ramif:
        return None
    if len(ramif) > 1: 
        print vid, genotype(vid), 'line ', g.node(vid)._line, len(ramif)
    return ramif[0]


for trunk in trunks:
    seq = []
    for vid in trunk:
        vars = []
        # Extract the next vertex that is the root of a sub-tree
        bid = filter_hamp_branch_crown(g, vid)
        if bid is None:
            vars = [-1]
        else:
            vars = [branching_type(bid, g)]

        inflo = is_inflo(g, bid) 
        branch_id = branching(g, vid)
        for name in variables:
            if inflo and name in inflo_variables:

                f = inflo_variables[name]
                vars.append(f(g.complex(bid)))
                print 'INFLO ', f(g.complex(bid))
            elif branch_id is not None: 
                f = variables[name]
                vars.append(f(branch_id))
            else:
                vars.append(0)

            #vars.extend([f(branching(g, vid)) if branching(g, vid) is not None elif inflo_variables[name] if is_inflo(g, vid) else 0 for f in variables.values()])
        vars.extend([f(vid) for f in global_variables.values()])
        seq.append(vars)
    final_seqs.append(seq)

#[[f(branching(g, vid)) if branching(g, vid) is not None else -1 for f in variables.values()] for vid in trunk] for trunk in trunks]
#final_seqs                          

In [ ]:
variables
variable_names = ['branching_type']+variables.keys()+global_variables.keys()
variable_names

In [ ]:
sequence_txt = analysis.write_sequences(final_seqs, variable_names, vertex_ids)
print sequence_txt